In [18]:
## for getting a known time of max light.
## uses .xls or .csv file generated using pandas dataframe.

import pandas as pd
import os
from pathlib import Path
print(os.getcwd())

/data/cartermg


In [35]:
# gets file
pathname = Path('msmts.csv')
df = pd.read_csv(pathname, sep='\t')

# reads values, gets time of max light
# PEAK MUST EXIST IN DURATION OF OBSERVATION
idxmax = df['rel_flux_T1'].idxmax()  
T0_BJDTBD = df.loc[idxmax, 'BJD_TDB']  # corresponding BJD_TDB

P = 0.13053502226702715    # in d. period calculated in 329 for V927. Needs to be edited if a different target.

print(f'Time of max light during OPO observation: {T0_BJDTBD}')

Time of max light during OPO observation: 2460761.946121


In [9]:
## for finding the target star coordinates, to be used in HJD and BJD conversion
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u

In [68]:
def simbad_coords(object_name):
    # query Simbad
    result = Simbad.query_object(object_name)
    if result is None:
        raise ValueError(f"Object '{object_name}' not found in Simbad.")
    
    ra = result['RA'][0]  # in 'hh mm ss' format
    dec = result['DEC'][0]  # in 'dd mm ss' format
    print(f'Star coordinates found: {ra}, {dec}')
    # convert to SkyCoord
    coord = SkyCoord(f"{ra} {dec}", unit=(u.hourangle, u.deg), frame='icrs')

    return coord

In [69]:
target_coords = simbad_coords('V927 Her')

Star coordinates found: 16 56 17.9985, +50 07 35.856


In [37]:
## sets observatory location.

obs_location = EarthLocation.from_geodetic(lon=-105.820417 * u.deg, lat=32.780361 * u.deg, height=2788 * u.m)
print(f"APO Earth Location Coordinates: {location}")

APO Earth Location Coordinates: (-1463998.7009965, -5166633.73441486, 3435013.97233452) m


In [53]:
## finds the phase.
## includes target coordinates from simbad_coords function.
## make sure to switch to HJD if that is the time used from the .csv file.

from astropy.io import fits

In [65]:
def calculate_phase(fits_file, T0, period, location, target_coord, T0_system='BJD'):
    """
    Parameters:
    - fits_file: path to the FITS file
    - T0: time of max light (in HJD or BJD depending on T0_system)
    - period: period of the star (days)
    - location: astropy EarthLocation object for the observatory
    - target_coord: astropy SkyCoord object for the star
    - T0_system: 'HJD' or 'BJD' depending on what T0 is
    """

    # open FITS file and read MJD
    hdul = fits.open(fits_file)
    header = hdul[0].header
    mjd_obs = header.get('AVE-MJD') or header.get('MJD')
    hdul.close()

    if mjd_obs is None:
        raise ValueError(f"No MJD keyword found in {fits_file}")

    # create a Time object from MJD
    time_obs = Time(mjd_obs, format='mjd', location=location)

    # calculate light travel time correction
    ltt_bary = time_obs.light_travel_time(target_coord, kind='barycentric')
    ltt_helio = time_obs.light_travel_time(target_coord, kind='heliocentric')

    # corrected times
    bjd_tdb = time_obs.tdb + ltt_bary
    hjd_utc = time_obs.utc + ltt_helio

    # choose which corrected time to use based on T0_system
    if T0_system.upper() == 'BJD':
        obs_corrected = bjd_tdb
    elif T0_system.upper() == 'HJD':
        obs_corrected = hjd_utc
    else:
        raise ValueError("T0_system must be either 'HJD' or 'BJD'.")

    # calculate phase
    phase = ((obs_corrected.value - T0) / period) % 1

    return phase, header.get('TIME'), header.get('DATE')

In [67]:
# pick a file name and run the final function.
filenames = ['V927Her1.merged.fits', 'V927Her2.merged.fits', 'V927Her3.merged.fits', 'V927Her4.merged.fits']

for filename in filenames:
    phase = calculate_phase(filename, T0_BJDTBD, P, obs_location, target_coords)
    print(f'Phase: {phase[0]}, at time {phase[1]} on {phase[2]}')

Phase: 0.10924263298511505, at time 12:40:49.252 on 2025-01-25
Phase: 0.12287905439734459, at time 12:43:23.042 on 2025-01-25
Phase: 0.13604092225432396, at time 12:45:51.480 on 2025-01-25
Phase: 0.14969028905034065, at time 12:48:25.416 on 2025-01-25
